<a href="https://colab.research.google.com/github/jake314jake/facial-recognition/blob/main/CNN%26SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
! sudo pip install mtcnn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.0 MB/s eta 0:00:00


In [ ]:
! unzip '/content/drive/MyDrive/0LFWtest.rar'  '/content/drive/MyDrive/0LFWtrain.rar'

Archive:  /content/drive/MyDrive/0LFWtest.rar
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/MyDrive/0LFWtest.rar or
        /content/drive/MyDrive/0LFWtest.rar.zip, and cannot find /content/drive/MyDrive/0LFWtest.rar.ZIP, period.


In [ ]:
!pip install  keras numpy


In [ ]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

# load images and extract faces for all images in a directory
def load_faces(directory):
    faces = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path)
        # store
        faces.append(face)
    return faces

# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for person: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

# load train dataset
dir_loc = input("Enter the path of the directory with training data: ")
dir_loc.replace('\\', '/')
dir_loc += '/'
trainX, trainy = load_dataset(dir_loc)
print(trainX.shape, trainy.shape)
# load test dataset
dir_loc = input("Enter the path of the directory with testing data: ")
dir_loc.replace('\\', '/')
dir_loc += '/'
testX, testy = load_dataset(dir_loc)
# save arrays to one file in compressed format
savez_compressed('init_face_array.npz', trainX, trainy, testX, testy)

In [ ]:
!pip install keras_facenet

  Preparing metadata (setup.py) ... done
  Created wheel for keras_facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10368 sha256=14b889c91011720ef87ade5c6831351bd20518dc9e961b4ec0293afe3caacc4f
  Stored in directory: /root/.cache/pip/wheels/1d/d8/a9/85cf04ea29321d2afcb82c0caaafdca9195385f9d68cbc7185
Successfully built keras_facenet


In [ ]:
from keras_facenet import FaceNet
model =FaceNet()
model

In [ ]:
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
#from numpy import asarray
#from numpy import savez_compressed
from keras.models import load_model

# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.embeddings(samples)
    return yhat[0]

# load the face dataset
data = load('init_face_array.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
# load the facenet model

# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)
# save arrays to one file in compressed format
savez_compressed('faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from random import choice
#from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot
# load dataset
data = load('faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# predict
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
#####################################################################################################################
testX_faces = data['arr_2']
# test model on a random example from the test dataset
selection = choice([i for i in range(testX.shape[0])])
#selection = 5
random_face_pixels = testX_faces[selection]
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])
# prediction for the face
samples = expand_dims(random_face_emb, axis=0)
yhat_class = model.predict(samples)
yhat_prob = model.predict_proba(samples)
# get name
class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
print('Expected: %s' % random_face_name[0])


#print the image with the predicted and expected output, please fork and help
'''print(random_face_pixels.shape)
pyplot.imshow(random_face_pixels)
title = '%s (%.3f)' % (predict_names[0], class_probability)
pyplot.title(title)
pyplot.show()'''

Loaded:  (960, 512) (960,) (480, 512) (480,)
Dataset: train=960, test=480
Accuracy: train=99.479, test=96.667
Predicted: Venus_Williams (13.115)
Expected: Venus_Williams


"print(random_face_pixels.shape)\npyplot.imshow(random_face_pixels)\ntitle = '%s (%.3f)' % (predict_names[0], class_probability)\npyplot.title(title)\npyplot.show()"

In [ ]:
!ls -l

total 92720
drwxr-xr-x 98 root root     4096 Feb 22 15:09 0LFWtest
drwxr-xr-x 98 root root     4096 Feb 22 15:09 0LFWtrain
drwxr-xr-x  2 root root     4096 Feb 22 21:24 data
drwx------  5 root root     4096 Feb 22 21:18 drive
-rw-r--r--  1 root root  2737382 Feb 22 22:16 faces-embeddings.npz
-rw-r--r--  1 root root 92184325 Feb 22 22:12 init_face_array.npz
drwxr-xr-x  1 root root     4096 Feb 21 14:21 sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!cp '/content/init_face_array.npz' '/content/drive/MyDrive'